# More Path Planning Techniques


### Categories of Path Planning Algorithms

1. **Grid-based Methods**: Discretize the environment into a grid (e.g., A*, Dijkstra).
2. **Sampling-based Methods**: Randomly sample the configuration space (e.g., RRT, PRM).
3. **Optimization-based Methods**: Formulate path planning as an optimization problem.
4. **Potential Field Methods**: Use artificial potential fields to guide the robot.

In this lab, we focus on **sampling-based methods** and **spanning tree approaches**, which are highly effective in high-dimensional and complex environments.

---



# **Importing Packages**

In [1]:
import numpy as np
import pygame
import random
from collections import deque
import math
import sys
import time 
from shapely.geometry import LineString, Point
import heapq


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
random.seed(42)
np.random.seed(42)

---

# **Rapidly-exploring Random Trees (RRT)**
## Conceptual Overview
Rapidly-exploring Random Trees (RRT) is a sampling-based path planning algorithm designed to efficiently search high-dimensional spaces. RRT incrementally builds a tree that spans the search space by randomly sampling points and extending the tree towards these samples. It is particularly effective in environments with complex obstacles and is suitable for real-time applications.

## Key Characteristics of RRT
- Exploration Efficiency: Quickly explores large, high-dimensional spaces.
- Incremental Growth: Builds the tree incrementally, allowing for real-time path planning.
- Probabilistic Completeness: Guarantees that a path will be found if one exists, given sufficient time.

## Algorithm Details
RRT operates through the following steps:
1. Initialization:
    - Start with a tree containing only the start node.
2. Sampling:
    - Randomly sample a point in the configuration space.
3. Nearest Neighbor:
    - Find the nearest node in the tree to the sampled point.
4. Steering:
    - Move from the nearest node towards the sampled point by a fixed step size.
5. Collision Checking:
    - Ensure that the path between the nearest node and the new node is free from obstacles.
6. Tree Expansion:
    - If the path is collision-free, add the new node to the tree.
7. Goal Checking:
    - If the new node is sufficiently close to the goal, connect it directly to the goal.
8. Iteration:
    - Repeat steps 2-7 until the goal is reached or a maximum number of iterations is exceeded.

In [3]:
class Node:
    def __init__(self, position):
        """
        Represents a node in the RRT.

        Parameters:
            position (tuple): The (x, y) coordinates of the node.
        """
        self.position = position
        self.parent = None

In [4]:
def distance(p1, p2):
    """
    Calculates the Euclidean distance between two points.

    Parameters:
        p1 (tuple): First point (x, y).
        p2 (tuple): Second point (x, y).

    Returns:
        float: Euclidean distance.
    """
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])


In [5]:
def get_random_point(rand_area):
    """
    Samples a random point within the defined sampling area.

    Parameters:
        rand_area (tuple): Sampling area (min, max) for both x and y.

    Returns:
        tuple: A randomly sampled (x, y) point.
    """
    return (random.uniform(rand_area[0], rand_area[1]),
            random.uniform(rand_area[0], rand_area[1]))


In [6]:
def get_nearest_node(tree, point):
    """
    Finds the nearest node in the tree to the given point.

    Parameters:
        tree (list): List of Node objects representing the tree.
        point (tuple): The (x, y) point to find the nearest node to.

    Returns:
        Node: The nearest Node in the tree.
    """
    return min(tree, key=lambda node: distance(node.position, point))


In [7]:
def steer(from_node, to_point, extend_length):
    """
    Steers from a node towards a point by a specified step size.

    Parameters:
        from_node (Node): The node from which to steer.
        to_point (tuple): The (x, y) point towards which to steer.
        extend_length (float): The step size.

    Returns:
        Node: The new node reached after steering.
    """
    theta = math.atan2(to_point[1] - from_node.position[1],
                       to_point[0] - from_node.position[0])
    new_pos = (from_node.position[0] + extend_length * math.cos(theta),
               from_node.position[1] + extend_length * math.sin(theta))
    new_pos = (int(round(new_pos[0])), int(round(new_pos[1])))
    return Node(new_pos)

In [8]:
def is_collision(p1, p2, obstacle_list, resolution):
    """
    Checks if the path between two points is collision-free.

    Parameters:
        p1 (tuple): Start point (x, y).
        p2 (tuple): End point (x, y).
        obstacle_list (list): List of obstacles, each defined as (x, y, radius).
        resolution (float): Sampling resolution along the path for collision checking.

    Returns:
        bool: True if there is a collision; False otherwise.
    """
    dist = distance(p1, p2)
    steps = int(dist / resolution)
    for i in range(steps + 1):
        x = p1[0] + (p2[0] - p1[0]) * i / steps
        y = p1[1] + (p2[1] - p1[1]) * i / steps
        for (ox, oy, r) in obstacle_list:
            if distance((x, y), (ox, oy)) <= r:
                return True  # Collision detected
    return False  # No collision

In [9]:
def extract_path(goal_node):
    """
    Extracts the path from the start node to the goal node.

    Parameters:
        goal_node (Node): The goal node with a parent chain back to the start node.

    Returns:
        list: Path as a list of (x, y) tuples from start to goal.
    """
    path = []
    node = goal_node
    while node is not None:
        path.append(node.position)
        node = node.parent
    return path[::-1]  # Reverse the path to start from the beginning

In [10]:
def draw_obstacles(obstacle_list):
    """
    Draws the obstacles on the Pygame screen.

    Parameters:
        obstacle_list (list): List of obstacles, each defined as (x, y, radius).
    """
    for (ox, oy, r) in obstacle_list:
        pygame.draw.circle(screen, GRAY, (int(ox), int(oy)), int(r))


def draw_tree(tree):
    """
    Draws the RRT tree on the Pygame screen.

    Parameters:
        tree (list): List of Node objects representing the tree.
    """
    for node in tree:
        if node.parent:
            pygame.draw.line(screen, BLUE, node.position, node.parent.position, 1)


def draw_path(path):
    """
    Draws the final path on the Pygame screen.

    Parameters:
        path (list): List of (x, y) tuples representing the path.
    """
    if path:
        pygame.draw.lines(screen, RED, False, path, 2)
        pygame.display.update()
        print("Path drawn!")
    else:
        print("No path to draw.")

In [11]:
def rrt_planning(start, goal, obstacle_list, rand_area, extend_dis=10.0, path_resolution=1.0, max_iter=5000):
    """
    Implements the Rapidly-exploring Random Tree (RRT) algorithm with Pygame visualization and timing.

    Parameters:
        start (tuple): Start position (x, y).
        goal (tuple): Goal position (x, y).
        obstacle_list (list): List of obstacles, each defined as (x, y, radius).
        rand_area (tuple): Sampling area (min, max) for both x and y.
        extend_dis (float): Step size for tree expansion.
        path_resolution (float): Resolution for collision checking along the path.
        max_iter (int): Maximum number of iterations to run the algorithm.

    Returns:
        list or None: Path from start to goal as a list of (x, y) tuples if found; otherwise, None.
    """
    start_node = Node(start)
    goal_node = Node(goal)
    tree = [start_node]

    start_time = time.time()

    for i in range(max_iter):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Sample a random point in the search space
        rnd = get_random_point(rand_area)

        # Find the nearest node in the tree to the sampled point
        nearest_node = get_nearest_node(tree, rnd)

        # Steer towards the sampled point from the nearest node
        new_node = steer(nearest_node, rnd, extend_dis)

        # Check if the path from the nearest node to the new node is collision-free
        if not is_collision(nearest_node.position, new_node.position, obstacle_list, path_resolution):
            new_node.parent = nearest_node
            tree.append(new_node)

            # Draw the new edge
            pygame.draw.line(screen, BLUE, nearest_node.position, new_node.position, 1)
            pygame.display.update()

            # Check if the new node is close enough to the goal
            if distance(new_node.position, goal) <= extend_dis:
                goal_node.parent = new_node
                tree.append(goal_node)
                path = extract_path(goal_node)

                # End timing
                end_time = time.time()
                elapsed_time = end_time - start_time

                return path, elapsed_time

        # Visualization updates
        screen.fill(WHITE)
        draw_obstacles(obstacle_list)
        draw_tree(tree)
        pygame.draw.circle(screen, GREEN, (int(goal[0]), int(goal[1])), 5)
        pygame.draw.circle(screen, RED, (int(start[0]), int(start[1])), 5)
        pygame.display.update()
        clock.tick(FPS)

    # End timing if path not found
    end_time = time.time()
    elapsed_time = end_time - start_time

    return None, elapsed_time  # Path not found within the maximum iterations

In [ ]:
pygame.init()
# Screen dimensions
WIDTH, HEIGHT = 800, 800  # Pixels
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("RRT Path Planning")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (200, 0, 0)
GREEN = (0, 200, 0)
BLUE = (0, 0, 200)
GRAY = (220, 220, 220)
YELLOW = (255, 255, 0)

clock = pygame.time.Clock()
FPS = 60



def main_rrt():
    obstacle_list = [
        (200, 200, 50),
        (300, 300, 75),
        (400, 400, 50),
        (500, 300, 75),
        (600, 200, 50),
        (400, 600, 75)
    ]

    start = (50, 50)
    goal = (750, 750)

    # Define sampling area
    rand_area = (0, WIDTH)

    screen.fill(WHITE)

    draw_obstacles(obstacle_list)
    pygame.draw.circle(screen, GREEN, (int(goal[0]), int(goal[1])), 5)
    pygame.draw.circle(screen, RED, (int(start[0]), int(start[1])), 5)
    pygame.display.update()

    print("Starting RRT Algorithm.")
    path, time_taken = rrt_planning(start, goal, obstacle_list, rand_area, extend_dis=10.0, path_resolution=1.0, max_iter=5000)

    if path:
        draw_path(path)
        print(f"Path found in {time_taken:.2f} seconds.")
    else:
        print(f"No path found. Time elapsed: {time_taken:.2f} seconds.")

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        clock.tick(FPS)



main_rrt()


## Detailed Analysis
1. Path Found:

    - If a path is found, it demonstrates the RRT's ability to explore the space and connect start to goal while avoiding obstacles.
    - The path may not be the shortest due to the nature of RRT's random exploration.
2. No Path Found:

    - If no path is found within the maximum iterations, consider increasing max_iter or adjusting expand_dis.
    - Another strategy is to bias the sampling towards the goal to increase the likelihood of reaching it.
3. Parameters Impact:

    - expand_dis:
        - Larger step sizes lead to faster exploration but may skip narrow passages.
        - Smaller step sizes allow for finer exploration but increase computation time.
    - max_iter:
        - Higher iterations provide more chances to find a path but increase computation time.
    - path_resolution:
        - Finer resolution ensures accurate collision checking but requires more computations.
4. Algorithm Strengths:

    - Efficient in high-dimensional spaces.
    - Capable of handling complex obstacle configurations.
    - Probabilistically complete, meaning it will find a path if one exists given enough time.
5. Algorithm Limitations:

    - Path optimality is not guaranteed; RRT paths can be suboptimal.
    - Sensitive to parameter settings, which require careful tuning.

---

# RRT*



## Introduction to RRT*

### What is RRT\*?

**Rapidly-exploring Random Trees Star (RRT\*)** is an extension of the standard Rapidly-exploring Random Trees (RRT) algorithm. While RRT is excellent for quickly finding feasible paths in high-dimensional spaces, it does not guarantee path optimality. RRT\*, introduced by Karaman and Frazzoli in 2011, addresses this limitation by ensuring that the paths generated are asymptotically optimal. This means that as the number of samples approaches infinity, the path cost converges to the optimal cost.

### RRT vs. RRT\*

| **Feature**               | **RRT**                                      | **RRT\***                                    |
|---------------------------|----------------------------------------------|----------------------------------------------|
| **Path Optimality**       | Does not guarantee optimal paths             | Guarantees asymptotic optimality             |
| **Tree Structure**        | Simple tree without considering cost         | Tree rewiring to optimize path costs         |
| **Computation Complexity**| Generally faster due to simpler structure    | Slightly higher due to additional rewiring steps |
| **Use Case**              | Real-time applications needing feasible paths| Applications requiring optimal paths         |

---

## Algorithm Details

### Key Features of RRT\*

1. **Cost-Based Node Selection**: Each node in the tree maintains the cost from the start node. When adding a new node, RRT\* considers the cheapest path to connect it.
2. **Rewiring**: After adding a new node, the algorithm checks neighboring nodes to see if their paths can be optimized through the new node.
3. **Asymptotic Optimality**: Guarantees that the path cost approaches the optimal cost as the number of samples increases.

### RRT\* Algorithm Steps

1. **Initialization**:
    - Start with a tree containing only the start node.
2. **Sampling**:
    - Randomly sample a point in the configuration space.
3. **Nearest Neighbor**:
    - Find the nearest node in the tree to the sampled point.
4. **Steering**:
    - Move from the nearest node towards the sampled point by a fixed step size.
5. **Collision Checking**:
    - Ensure that the path between the nearest node and the new node is free from obstacles.
6. **Near Nodes Identification**:
    - Identify all nodes within a certain radius of the new node.
7. **Choose Parent with Minimum Cost**:
    - Select the node among the near nodes that offers the lowest cost path to the new node.
8. **Add New Node**:
    - Add the new node to the tree with the selected parent.
9. **Rewiring**:
    - For each near node, check if the path through the new node offers a lower cost. If so, update the parent of the near node to the new node.
10. **Goal Checking**:
    - If the new node is sufficiently close to the goal, and the path is collision-free, connect it to the goal.
11. **Iteration**:
    - Repeat steps 2-10 until the goal is reached or a maximum number of iterations is exceeded.

---

## RRT* Implementation in Pygame

### Code Overview

The following implementation of RRT\* in Pygame includes:

- **Grid Environment**: Defines the navigable space and obstacles.
- **RRT\* Algorithm**: Implements the RRT\* algorithm with cost calculations and rewiring.
- **Visualization**: Uses Pygame to dynamically visualize the tree growth, obstacle avoidance, and the final optimal path.

In [13]:
class RRTStarNode:
    def __init__(self, position):
        """
        Represents a node in the RRT* tree.

        Parameters:
            position (tuple): The (x, y) coordinates of the node.
        """
        self.position = position
        self.parent = None
        self.cost = 0.0  # Cost from start node

In [14]:
# Define the grid environment
class Grid:
    def __init__(self, width, height, obstacle_list):
        """
        Represents a grid-based environment.

        Parameters:
            width (int): Width of the grid.
            height (int): Height of the grid.
            obstacle_list (list): List of obstacles, each defined as (x, y, radius).
        """
        self.width = width
        self.height = height
        self.obstacles = obstacle_list
        self.grid = np.zeros((width, height), dtype=int)
        self.create_obstacles()
    
    def create_obstacles(self):
        """
        Marks grid cells as obstacles based on the obstacle list.
        """
        for (ox, oy, r) in self.obstacles:
            for x in range(max(0, ox - r), min(self.width, ox + r + 1)):
                for y in range(max(0, oy - r), min(self.height, oy + r + 1)):
                    if distance((x, y), (ox, oy)) <= r:
                        self.grid[x][y] = 1  # Mark as obstacle
    
    def is_free(self, node):
        """
        Checks if a node is free (not an obstacle).

        Parameters:
            node (tuple): The (x, y) coordinates of the node.

        Returns:
            bool: True if the node is free; False otherwise.
        """
        x, y = node
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.grid[x][y] == 0
        return False


In [15]:
def distance(p1, p2):
    """
    Calculates the Euclidean distance between two points.

    Parameters:
        p1 (tuple): First point (x, y).
        p2 (tuple): Second point (x, y).

    Returns:
        float: Euclidean distance.
    """
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

In [16]:
def get_random_point(rand_area):
    """
    Samples a random point within the defined sampling area.

    Parameters:
        rand_area (tuple): Sampling area (min, max) for both x and y.

    Returns:
        tuple: A randomly sampled (x, y) point.
    """
    return (random.uniform(rand_area[0], rand_area[1]),
            random.uniform(rand_area[0], rand_area[1]))

In [17]:
def get_nearest_node(tree, point):
    """
    Finds the nearest node in the tree to the given point.

    Parameters:
        tree (list): List of RRTStarNode objects representing the tree.
        point (tuple): The (x, y) point to find the nearest node to.

    Returns:
        RRTStarNode: The nearest Node in the tree.
    """
    return min(tree, key=lambda node: distance(node.position, point))


In [18]:
def steer(from_node, to_point, extend_length):
    """
    Steers from a node towards a point by a specified step size.

    Parameters:
        from_node (RRTStarNode): The node from which to steer.
        to_point (tuple): The (x, y) point towards which to steer.
        extend_length (float): The step size.

    Returns:
        RRTStarNode: The new node reached after steering.
    """
    theta = math.atan2(to_point[1] - from_node.position[1],
                       to_point[0] - from_node.position[0])
    new_pos = (from_node.position[0] + extend_length * math.cos(theta),
               from_node.position[1] + extend_length * math.sin(theta))
    new_pos = (int(round(new_pos[0])), int(round(new_pos[1])))
    return RRTStarNode(new_pos)

In [19]:
def collision_free(p1, p2, grid):
    """
    Checks if the path between two points is free from obstacles using Bresenham's line algorithm.

    Parameters:
        p1 (tuple): Start point (x, y).
        p2 (tuple): End point (x, y).
        grid (Grid): The grid environment.

    Returns:
        bool: True if the path is collision-free; False otherwise.
    """
    x1, y1 = p1
    x2, y2 = p2
    dx = x2 - x1
    dy = y2 - y1
    steps = max(abs(dx), abs(dy))
    if steps == 0:
        return grid.is_free(p1)
    x_inc = dx / steps
    y_inc = dy / steps
    x, y = x1, y1
    for _ in range(int(steps)):
        if not grid.is_free((int(round(x)), int(round(y)))):
            return False
        x += x_inc
        y += y_inc
    return True

In [20]:
def get_near_nodes(new_node, tree, radius):
    """
    Finds all nodes in the tree within a certain radius of the new node.

    Parameters:
        new_node (RRTStarNode): The new node.
        tree (list): List of existing nodes in the tree.
        radius (float): Radius to search for near nodes.

    Returns:
        list: List of near nodes.
    """
    near_nodes = []
    for node in tree:
        if distance(node.position, new_node.position) <= radius:
            near_nodes.append(node)
    return near_nodes

In [21]:
def rrt_star_planning(start, goal, obstacle_list, rand_area, grid, extend_length=10.0, path_resolution=1.0, max_iter=5000):
    """
    Implements the RRT* algorithm with Pygame visualization and timing.

    Parameters:
        start (tuple): Start position (x, y).
        goal (tuple): Goal position (x, y).
        obstacle_list (list): List of obstacles, each defined as (x, y, radius).
        rand_area (tuple): Sampling area (min, max) for both x and y.
        grid (Grid): The grid environment.
        extend_length (float): Step size for tree expansion.
        path_resolution (float): Resolution for collision checking along the path.
        max_iter (int): Maximum number of iterations to run the algorithm.

    Returns:
        list or None: Path from start to goal as a list of (x, y) tuples if found; otherwise, None.
    """
    start_node = RRTStarNode(start)
    tree = [start_node]
    goal_node = None
    found_goal = False
    search_radius = extend_length * 2.0  # Adjust based on environment size

    # Start timing
    start_time = time.time()

    for i in range(max_iter):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Sample a random point in the search space
        rnd = get_random_point(rand_area)

        # Find the nearest node in the tree to the sampled point
        nearest_node = get_nearest_node(tree, rnd)

        # Steer towards the sampled point from the nearest node
        new_node = steer(nearest_node, rnd, extend_length)

        # Check if the path from the nearest node to the new node is collision-free
        if not collision_free(nearest_node.position, new_node.position, grid):
            continue

        # Find near nodes within the search radius
        near_nodes = get_near_nodes(new_node, tree, search_radius)

        # Choose the parent with the minimum cost
        min_cost = nearest_node.cost + distance(nearest_node.position, new_node.position)
        best_parent = nearest_node

        for near_node in near_nodes:
            if collision_free(near_node.position, new_node.position, grid):
                cost = near_node.cost + distance(near_node.position, new_node.position)
                if cost < min_cost:
                    min_cost = cost
                    best_parent = near_node

        # Set the best parent for the new node
        new_node.parent = best_parent
        new_node.cost = min_cost
        tree.append(new_node)

        # Rewire the tree around the new node
        for near_node in near_nodes:
            if collision_free(new_node.position, near_node.position, grid):
                cost_through_new = new_node.cost + distance(new_node.position, near_node.position)
                if cost_through_new < near_node.cost:
                    near_node.parent = new_node
                    near_node.cost = cost_through_new

                    pygame.draw.line(screen, ORANGE, new_node.position, near_node.position, 1)
                    pygame.display.update()

        pygame.draw.line(screen, BLUE, best_parent.position, new_node.position, 1)
        pygame.display.update()

        # Check if the new node is close enough to the goal
        if distance(new_node.position, goal) <= extend_length:
            if collision_free(new_node.position, goal, grid):
                goal_node = RRTStarNode(goal)
                goal_node.parent = new_node
                goal_node.cost = new_node.cost + distance(new_node.position, goal)
                tree.append(goal_node)

                pygame.draw.line(screen, GREEN, new_node.position, goal_node.position, 2)
                pygame.display.update()
                found_goal = True
                print(f"Goal reached at iteration {i}")
                break

        clock.tick(FPS)

    end_time = time.time()
    elapsed_time = end_time - start_time

    if found_goal:
        # Reconstruct the path from start to goal
        path = extract_path(goal_node)
        return path, elapsed_time  
    else:
        print("No path found within the maximum iterations.")
        return None, elapsed_time


In [22]:
def extract_path(goal_node):
    """
    Extracts the path from the start node to the goal node.

    Parameters:
        goal_node (Node): The goal node with a parent chain back to the start node.

    Returns:
        list: Path as a list of (x, y) tuples from start to goal.
    """
    path = []
    node = goal_node
    while node is not None:
        path.append(node.position)
        node = node.parent
    return path[::-1]  # Reverse the path to start from the beginning


In [23]:
def draw_obstacles(obstacle_list):
    """
    Draws the obstacles on the Pygame screen.

    Parameters:
        obstacle_list (list): List of obstacles, each defined as (x, y, radius).
    """
    for (ox, oy, r) in obstacle_list:
        pygame.draw.circle(screen, GRAY, (int(ox), int(oy)), int(r))

def draw_tree(tree):
    """
    Draws the RRT* tree on the Pygame screen.

    Parameters:
        tree (list): List of RRTStarNode objects representing the tree.
    """
    for node in tree:
        if node.parent:
            pygame.draw.line(screen, BLUE, node.position, node.parent.position, 1)

def draw_path(path):
    """
    Draws the final path on the Pygame screen.

    Parameters:
        path (list): List of (x, y) tuples representing the path.
    """
    if path:
        pygame.draw.lines(screen, YELLOW, False, path, 3)
        pygame.display.update()
        print("Path drawn!")
    else:
        print("No path to draw.")

In [ ]:
pygame.init()

GRID_WIDTH, GRID_HEIGHT = 800, 800
SCALE = 10  # Scale factor: pixels per grid unit
SCREEN_WIDTH, SCREEN_HEIGHT = GRID_WIDTH, GRID_HEIGHT
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("RRT* Path Planning")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (200, 0, 0)
GREEN = (0, 200, 0)
BLUE = (0, 0, 255)
GRAY = (220, 220, 220)
YELLOW = (255, 255, 0)
ORANGE = (255, 165, 0)

clock = pygame.time.Clock()
FPS = 60

def main_rrt_star():
    obstacle_list = [
        (200, 200, 50),
        (300, 300, 75),
        (400, 400, 50),
        (500, 300, 75),
        (600, 200, 50),
        (400, 600, 75)
    ]

    start = (50, 50)
    goal = (750, 750)

    rand_area = (0, SCREEN_WIDTH)

    screen.fill(WHITE)

    draw_obstacles(obstacle_list)
    pygame.draw.circle(screen, GREEN, (int(goal[0]), int(goal[1])), 5)
    pygame.draw.circle(screen, RED, (int(start[0]), int(start[1])), 5)
    pygame.display.update()

    grid = Grid(SCREEN_WIDTH, SCREEN_HEIGHT, obstacle_list)

    print("Starting RRT* Algorithm.")
    path, time_taken = rrt_star_planning(start, goal, obstacle_list, rand_area, grid, extend_length=15.0, path_resolution=1.0, max_iter=5000)

    if path:
        draw_path(path)
        print(f"Path found in {time_taken:.2f} seconds.")
    else:
        print(f"No path found. Time elapsed: {time_taken:.2f} seconds.")

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        clock.tick(FPS)

if __name__ == "__main__":
    main_rrt_star()


---

# Spanning Trees
## Conceptual Overview
A spanning tree is a subset of a graph that connects all the vertices together without any cycles and with the minimum possible number of edges. In the context of path planning, spanning trees help in exploring all possible paths within a graph representation of the environment, ensuring comprehensive coverage of the navigable space.
## Key Characteristics of Spanning Trees
- Connectivity: Ensures all nodes (positions) are connected.
- Acyclic: Contains no cycles, preventing redundant paths.
- Minimality: For n nodes, a spanning tree has exactly n-1 edges.

## 1. Depth-First Spanning Tree

### **Definition**
A **Depth-First Spanning Tree** is a spanning tree constructed using the Depth-First Search (DFS) algorithm. Starting from a root node, DFS explores as far as possible along each branch before backtracking.

### **Properties**
- **Deep Exploration**: Nodes are added to the tree by exploring each branch to its deepest point before backtracking.
- **Path Characteristics**: Paths from the root to any node can be long and winding, depending on the graph's structure.
- **Cycle Detection**: DFS is effective in detecting cycles within graphs.

### **Applications**
- **Maze Solving**: DFS is used to explore all possible paths in maze-solving algorithms.
- **Topological Sorting**: In directed acyclic graphs, DFS helps in ordering tasks.
- **Cycle Detection**: Identifying cycles in graphs for various computational problems.

### Steps of DFS-based Spanning Tree Construction
1. Initialization:
    - Start with a root node (e.g., start position).
    - Initialize an empty tree structure to store parent-child relationships.
2. Traversal:
    - Visit an adjacent unvisited node and mark it as part of the tree.
    - Recursively apply DFS to the new node.
3. Backtracking:
    - If a node has no unvisited neighbors, backtrack to the previous node.
4. Termination:
    - The process continues until all reachable nodes are visited and included in the spanning tree.

We'll implement a simple environment and generate a spanning tree using DFS

In [8]:
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 800
FPS = 60
OBSTACLE_COUNT = 20
OBSTACLE_RADIUS = 30
ROBOT_RADIUS = 10
STEP_SIZE = 20
GOAL_THRESHOLD = 15  # Distance threshold to consider the goal reached

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREY = (200, 200, 200)
BLUE = (0, 0, 255)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
ORANGE = (255, 165, 0)
PURPLE = (160, 32, 240)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Spanning Tree Path Planning Simulation")
clock = pygame.time.Clock()

def distance(p1, p2):
    return math.hypot(p2[0]-p1[0], p2[1]-p1[1])

def line_intersects_obstacles(p1, p2, obstacles):
    line = LineString([p1, p2])
    for obs in obstacles:
        circle = Point(obs).buffer(OBSTACLE_RADIUS)
        if line.intersects(circle):
            return True
    return False

def get_random_point():
    return (random.randint(50, WIDTH-50), random.randint(50, HEIGHT-50))

def draw_obstacles(obstacles):
    for obs in obstacles:
        pygame.draw.circle(screen, BLACK, obs, OBSTACLE_RADIUS)

def draw_robot(position):
    pygame.draw.circle(screen, BLUE, (int(position[0]), int(position[1])), ROBOT_RADIUS)

def draw_goal(goal):
    pygame.draw.circle(screen, RED, (int(goal[0]), int(goal[1])), ROBOT_RADIUS)

def draw_start(start):
    pygame.draw.circle(screen, GREEN, (int(start[0]), int(start[1])), ROBOT_RADIUS)

def draw_spanning_tree(edges):
    for edge in edges:
        pygame.draw.line(screen, GREY, edge[0], edge[1], 2)

def draw_path(path):
    if len(path) > 1:
        pygame.draw.lines(screen, ORANGE, False, path, 4)

class Simulation:
    def __init__(self):
        self.obstacles = self.generate_obstacles()
        self.start = self.get_user_point("Select Start Position")
        self.goal = self.get_user_point("Select Goal Position")
        self.edges = []
        self.visited = set()
        self.stack = []
        self.path = []
        self.parent = {}
        self.finished = False
        self.found_goal = False

        # Initialize DFS stack with the start node
        self.stack.append(self.start)
        self.visited.add(self.start)

    def generate_obstacles(self):
        obstacles = []
        for _ in range(OBSTACLE_COUNT):
            while True:
                point = get_random_point()
                if distance(point, (WIDTH//2, HEIGHT//2)) > OBSTACLE_RADIUS + 50:
                    # Ensure obstacles do not overlap with existing obstacles
                    overlap = False
                    for obs in obstacles:
                        if distance(point, obs) < 2 * OBSTACLE_RADIUS:
                            overlap = True
                            break
                    if not overlap:
                        obstacles.append(point)
                        break
        return obstacles

    def get_user_point(self, message):
        font = pygame.font.SysFont(None, 36)
        selected = False
        point = None
        while not selected:
            screen.fill(WHITE)
            text = font.render(message, True, BLACK)
            screen.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2 - text.get_height()//2 - 50))
            instruction = font.render("Click to set position", True, BLACK)
            screen.blit(instruction, (WIDTH//2 - instruction.get_width()//2, HEIGHT//2 - instruction.get_height()//2))
            draw_obstacles(self.obstacles)
            pygame.display.flip()
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    # Ensure the point is not inside an obstacle
                    collision = False
                    for obs in self.obstacles:
                        if distance(pos, obs) < OBSTACLE_RADIUS + ROBOT_RADIUS:
                            collision = True
                            break
                    if not collision:
                        point = pos
                        selected = True
        return point

    def run_dfs_step(self):
        if not self.stack:
            self.finished = True
            return
        current = self.stack.pop()
        # Check if current node is within the goal threshold
        if distance(current, self.goal) < GOAL_THRESHOLD:
            self.found_goal = True
            # Connect the current node to the goal
            self.edges.append((current, self.goal))
            self.parent[self.goal] = current
            self.construct_path()
            return
        neighbors = self.get_neighbors(current)
        random.shuffle(neighbors)  # Randomize exploration for varied spanning trees
        for neighbor in neighbors:
            if neighbor not in self.visited:
                if not line_intersects_obstacles(current, neighbor, self.obstacles):
                    self.edges.append((current, neighbor))
                    self.parent[neighbor] = current
                    self.stack.append(neighbor)
                    self.visited.add(neighbor)

    def get_neighbors(self, current):
        # Generate neighbors in 8 directions
        dirs = [(-STEP_SIZE, -STEP_SIZE), (-STEP_SIZE, 0), (-STEP_SIZE, STEP_SIZE),
                (0, -STEP_SIZE),                    (0, STEP_SIZE),
                (STEP_SIZE, -STEP_SIZE),  (STEP_SIZE, 0), (STEP_SIZE, STEP_SIZE)]
        neighbors = []
        for dx, dy in dirs:
            new_x = current[0] + dx
            new_y = current[1] + dy
            if 0 <= new_x < WIDTH and 0 <= new_y < HEIGHT:
                neighbors.append((new_x, new_y))
        return neighbors

    def construct_path(self):
        self.path = []
        current = self.goal
        while current != self.start:
            self.path.append(current)
            current = self.parent.get(current)
            if current is None:
                # No path found
                self.path = []
                self.found_goal = False
                return
        self.path.append(self.start)
        self.path.reverse()

    def update(self):
        if not self.found_goal and not self.finished:
            self.run_dfs_step()

    def draw(self):
        screen.fill(WHITE)
        draw_obstacles(self.obstacles)
        draw_spanning_tree(self.edges)
        draw_start(self.start)
        draw_goal(self.goal)
        if self.found_goal:
            draw_path(self.path)
        elif self.finished:
            font = pygame.font.SysFont(None, 48)
            text = font.render("No Path Found", True, RED)
            screen.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2 - text.get_height()//2))
        pygame.display.flip()

    def visualize_robot(self):
        if self.found_goal and self.path:
            for point in self.path:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                self.draw()
                draw_robot(point)
                pygame.display.flip()
                clock.tick(60)  # Control the speed of the robot
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()

    def run(self):
        while True:
            clock.tick(FPS)
            self.update()
            self.draw()
            if self.found_goal:
                self.visualize_robot()
            if self.finished and not self.found_goal:
                # If DFS is finished and no path found, wait for user to close
                pass
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

# Run the simulation
if __name__ == "__main__":
    sim = Simulation()
    sim.run()


SystemExit: 

---

### 2. Minimum Spanning Tree (MST)

#### **Definition**
A **Minimum Spanning Tree** of a weighted, connected, undirected graph is a spanning tree whose sum of edge weights is minimized. In other words, it connects all the vertices together with the least possible total edge weight.

#### **Properties**
- **Uniqueness**: If all edge weights are distinct, the MST is unique. Otherwise, there may be multiple MSTs with the same total weight.
- **No Cycles**: Being a spanning tree, it contains no cycles.
- **Connectivity**: It connects all the vertices in the graph.

#### **Algorithms to Find MST**
- **Kruskal's Algorithm**: Builds the MST by adding the next smallest edge that doesn't form a cycle:
    - Sort All Edges: Arrange all edges in non-decreasing order of their weights.
    - Initialize: Start with an empty tree (no edges).
    - Add Edges: Iterate through the sorted edges and add the smallest edge to the tree if it doesn't form a cycle.
    - Repeat: Continue until all vertices are connected.
- **Prim's Algorithm**: Starts from an arbitrary node and grows the MST by adding the smallest edge connecting the tree to a new vertex.
    - Initialize: Start with a single node (the "start" node) and mark it as part of the MST.
    - Select Minimum Edge: From the edges connected to the nodes in the MST, select the edge with the smallest weight that connects to a node not yet in the MST.
    - Add to MST: Add the selected edge and the connected node to the MST.
    - Repeat: Continue selecting the smallest edge that connects the MST to a new node until all nodes are included.

#### **Applications**
- **Network Design**: Designing least-cost networks such as computer, telecommunication, and transportation networks.
- **Approximation Algorithms**: Serving as a foundation for algorithms solving more complex problems.
- **Clustering**: Used in hierarchical clustering methods.

In [16]:
import pygame
import sys
import math
import random
from shapely.geometry import LineString, Point
from collections import deque
import heapq

pygame.init()

# Constants
WIDTH, HEIGHT = 800, 800
FPS = 60
OBSTACLE_COUNT = 20
OBSTACLE_RADIUS = 30
ROBOT_RADIUS = 10
STEP_SIZE = 20
GOAL_THRESHOLD = 15  # Distance threshold to consider the goal reached
SAMPLE_COUNT = 100  # Number of sample points for MST

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREY = (200, 200, 200)
BLUE = (0, 0, 255)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
ORANGE = (255, 165, 0)
PURPLE = (160, 32, 240)
MST_COLOR = (150, 150, 150)  # Lighter grey for MST edges
PATH_COLOR = (255, 100, 0)    # Bright orange for the final path

# Initialize Pygame screen and clock
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Real-Time MST Path Planning Simulation")
clock = pygame.time.Clock()

def distance(p1, p2):
    """Calculate Euclidean distance between two points."""
    return math.hypot(p2[0]-p1[0], p2[1]-p1[1])

def line_intersects_obstacles(p1, p2, obstacles):
    """Check if the line between p1 and p2 intersects any obstacle."""
    line = LineString([p1, p2])
    for obs in obstacles:
        circle = Point(obs).buffer(OBSTACLE_RADIUS)
        if line.intersects(circle):
            return True
    return False

def get_random_point():
    """Generate a random point within the screen boundaries with a margin."""
    return (random.randint(50, WIDTH-50), random.randint(50, HEIGHT-50))

def draw_obstacles(obstacles):
    """Draw all obstacles on the screen."""
    for obs in obstacles:
        pygame.draw.circle(screen, BLACK, obs, OBSTACLE_RADIUS)

def draw_robot(position):
    """Draw the robot at the given position."""
    pygame.draw.circle(screen, BLUE, (int(position[0]), int(position[1])), ROBOT_RADIUS)

def draw_goal(goal):
    """Draw the goal position."""
    pygame.draw.circle(screen, RED, (int(goal[0]), int(goal[1])), ROBOT_RADIUS)

def draw_start(start):
    """Draw the start position."""
    pygame.draw.circle(screen, GREEN, (int(start[0]), int(start[1])), ROBOT_RADIUS)

def draw_graph(edges, color=MST_COLOR):
    """Draw edges of the graph (e.g., MST)."""
    for edge in edges:
        pygame.draw.line(screen, color, edge[0], edge[1], 2)

def draw_path(path):
    """Draw the final path from start to goal."""
    if len(path) > 1:
        pygame.draw.lines(screen, PATH_COLOR, False, path, 4)

class Simulation:
    def __init__(self):
        """Initialize the simulation."""
        self.obstacles = self.generate_obstacles()
        self.start = self.get_user_point("Select Start Position")
        self.goal = self.get_user_point("Select Goal Position")
        self.samples = self.generate_samples()
        self.graph = {}  # Adjacency list {point: [(neighbor, distance), ...], ...}
        self.all_possible_edges = []
        self.construct_graph()
        
        # Prim's algorithm initialization
        self.mst_edges = []
        self.mst_finished = False
        self.mst_heap = []
        self.visited_mst = set()
        self.parent = {}
        
        # Start Prim's algorithm
        self.initialize_prim()
        
        # Path variables
        self.path = []
        self.path_found = False
        self.path_computed = False

    def generate_obstacles(self):
        """Generate non-overlapping obstacles randomly placed on the screen."""
        obstacles = []
        for _ in range(OBSTACLE_COUNT):
            attempts = 0
            while attempts < 100:
                point = get_random_point()
                if distance(point, (WIDTH//2, HEIGHT//2)) > OBSTACLE_RADIUS + 50:
                    # Ensure obstacles do not overlap with existing obstacles
                    overlap = False
                    for obs in obstacles:
                        if distance(point, obs) < 2 * OBSTACLE_RADIUS:
                            overlap = True
                            break
                    if not overlap:
                        obstacles.append(point)
                        break
                attempts += 1
        return obstacles

    def get_user_point(self, message):
        """
        Prompt the user to click on the screen to set a point (start or goal).
        Ensures the point is not inside any obstacle.
        """
        font = pygame.font.SysFont(None, 36)
        selected = False
        point = None
        while not selected:
            screen.fill(WHITE)
            draw_obstacles(self.obstacles)
            text = font.render(message, True, BLACK)
            screen.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2 - text.get_height()//2 - 50))
            instruction = font.render("Click to set position", True, BLACK)
            screen.blit(instruction, (WIDTH//2 - instruction.get_width()//2, HEIGHT//2 - instruction.get_height()//2))
            pygame.display.flip()
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    # Ensure the point is not inside an obstacle
                    collision = False
                    for obs in self.obstacles:
                        if distance(pos, obs) < OBSTACLE_RADIUS + ROBOT_RADIUS:
                            collision = True
                            break
                    if not collision:
                        point = pos
                        selected = True
        return point

    def generate_samples(self):
        """Generate sample points for the MST, including start and goal."""
        samples = []
        samples.append(self.start)
        samples.append(self.goal)
        while len(samples) < SAMPLE_COUNT + 2:
            point = get_random_point()
            collision = False
            for obs in self.obstacles:
                if distance(point, obs) < OBSTACLE_RADIUS + ROBOT_RADIUS:
                    collision = True
                    break
            if not collision:
                samples.append(point)
        return samples

    def construct_graph(self):
        """Construct the adjacency list graph by connecting sample points without crossing obstacles."""
        self.graph = {point: [] for point in self.samples}
        for i in range(len(self.samples)):
            for j in range(i+1, len(self.samples)):
                p1 = self.samples[i]
                p2 = self.samples[j]
                if not line_intersects_obstacles(p1, p2, self.obstacles):
                    dist = distance(p1, p2)
                    self.graph[p1].append((p2, dist))
                    self.graph[p2].append((p1, dist))
                    self.all_possible_edges.append((p1, p2, dist))
        print(f"Total connected edges: {len(self.all_possible_edges)}")

    def initialize_prim(self):
        """Initialize Prim's algorithm by adding the start node and its edges to the heap."""
        # Start with the start point
        self.visited_mst.add(self.start)
        for neighbor, dist in self.graph[self.start]:
            heapq.heappush(self.mst_heap, (dist, self.start, neighbor))
        print("Prim's algorithm initialized.")

    def step_prim(self):
        """
        Perform one step of Prim's algorithm.
        Adds the smallest edge that connects a new node to the MST.
        """
        if self.mst_heap and not self.mst_finished:
            dist, from_node, to_node = heapq.heappop(self.mst_heap)
            if to_node not in self.visited_mst:
                self.visited_mst.add(to_node)
                self.mst_edges.append((from_node, to_node))
                self.parent[to_node] = from_node
                # Add new edges from the newly visited node
                for neighbor, ndist in self.graph[to_node]:
                    if neighbor not in self.visited_mst:
                        heapq.heappush(self.mst_heap, (ndist, to_node, neighbor))
                
                # **Check if the goal has been connected**
                if to_node == self.goal:
                    self.mst_finished = True
                    print("Goal connected. Prim's algorithm completed.")
                    self.find_path()
        
        # **Handle the case where the heap is empty but the goal hasn't been connected**
        elif not self.mst_heap and not self.mst_finished:
            self.mst_finished = True
            print("Prim's algorithm completed. No path found.")
            self.find_path()

    def find_path(self):
        """
        Reconstruct the path from start to goal using BFS on the MST.
        Sets the path_found flag accordingly.
        """
        # Build adjacency list for MST
        adj = {}
        for edge in self.mst_edges:
            adj.setdefault(edge[0], []).append(edge[1])
            adj.setdefault(edge[1], []).append(edge[0])
        
        queue = deque()
        queue.append(self.start)
        visited = {self.start: None}
        while queue:
            current = queue.popleft()
            if current == self.goal:
                break
            for neighbor in adj.get(current, []):
                if neighbor not in visited:
                    visited[neighbor] = current
                    queue.append(neighbor)
        
        # Reconstruct path
        if self.goal in visited:
            path = []
            current = self.goal
            while current != self.start:
                path.append(current)
                current = visited[current]
            path.append(self.start)
            path.reverse()
            self.path = path
            self.path_found = True
            print("Path found!")
        else:
            self.path = []
            self.path_found = False
            print("No path found.")

    def draw(self):
        """Render all elements on the screen."""
        screen.fill(WHITE)
        draw_obstacles(self.obstacles)
        draw_graph(self.mst_edges, MST_COLOR)
        draw_start(self.start)
        draw_goal(self.goal)
        if self.path_found:
            draw_path(self.path)
        elif self.mst_finished and not self.path_found:
            font = pygame.font.SysFont(None, 48)
            text = font.render("No Path Found", True, RED)
            screen.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2 - text.get_height()//2))
        pygame.display.flip()

    def visualize_robot(self):
        """
        Animate the robot moving along the found path.
        This function enters its own loop to handle the animation.
        """
        if self.path_found and self.path:
            for point in self.path:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                self.draw()
                draw_robot(point)
                pygame.display.flip()
                pygame.time.delay(100)  # Adjust speed as needed
            # After animation, keep displaying the final state
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()

    def run(self):
        """Run the main simulation loop."""
        while True:
            clock.tick(FPS)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
            
            if not self.mst_finished:
                self.step_prim()
            elif not self.path_computed:
                self.path_computed = True
                if self.path_found:
                    self.visualize_robot()
                # If no path is found, just continue to display the message
            
            self.draw()


sim = Simulation()
sim.run()


Total connected edges: 2634
Prim's algorithm initialized.
Goal connected. Prim's algorithm completed.
Path found!


SystemExit: 

---

## Other Types of Spanning Trees

Spanning trees are fundamental structures in graph theory with various applications in computer science, network design, and robotics. Below are detailed explanations of different types of spanning trees:



### 2. Maximum Spanning Tree

#### **Definition**
A **Maximum Spanning Tree** of a weighted, connected, undirected graph is a spanning tree whose sum of edge weights is maximized. It connects all the vertices together with the highest possible total edge weight.

#### **Properties**
- **Uniqueness**: Similar to MST, if all edge weights are distinct, the Maximum Spanning Tree is unique.
- **No Cycles**: It contains no cycles.
- **Connectivity**: It connects all the vertices in the graph.

#### **Algorithms to Find Maximum Spanning Tree**
- **Kruskal's Algorithm**: Modified to select edges in decreasing order of weight instead of increasing.
- **Prim's Algorithm**: Modified to choose the maximum weight edge connecting the tree to a new vertex.

#### **Applications**
- **Maximum Bandwidth Path**: Finding paths with the highest possible bandwidth in networks.
- **Clustering**: Forming clusters based on the strongest connections.
- **Competitive Networking**: Designing networks that can handle maximum traffic.

### 5. Rooted Spanning Tree

#### **Definition**
A **Rooted Spanning Tree** is a spanning tree in which one vertex is designated as the root. All edges are directed away from (or towards) the root, establishing a hierarchical structure.

#### **Properties**
- **Hierarchy**: Establishes a parent-child relationship among nodes.
- **Traversal**: Facilitates tree traversal methods like Depth-First Search (DFS) and Breadth-First Search (BFS) starting from the root.
- **Single Entry Point**: All paths originate from the root.

#### **Applications**
- **Organizational Structures**: Representing hierarchies in organizations.
- **Routing Protocols**: In computer networks, protocols like Spanning Tree Protocol (STP) use rooted spanning trees to prevent loops.
- **Data Structures**: Implementing trees in computer science, such as file directory structures.

### 6. Breadth-First Spanning Tree

#### **Definition**
A **Breadth-First Spanning Tree** is a spanning tree constructed using the Breadth-First Search (BFS) algorithm. Starting from a root node, BFS explores all neighbors at the present depth before moving on to nodes at the next depth level.

#### **Properties**
- **Level-wise Exploration**: Nodes are added to the tree in order of their distance from the root.
- **Shortest Path**: For unweighted graphs, the BFS spanning tree contains the shortest paths from the root to all other nodes.
- **Layered Structure**: The tree exhibits a layered structure based on node depth.

#### **Applications**
- **Shortest Path Finding**: Useful in unweighted graphs to determine the shortest path from a source to all other nodes.
- **Broadcasting**: In networking, used to broadcast messages efficiently.
- **Social Networking**: Analyzing connections and degrees of separation.

---

Now you try to implement the spanning tree with BFS:

In [ ]:
# your code here


---

# Probabilistic Roadmaps (PRM)
## Conceptual Overview
Probabilistic Roadmaps (PRM) are a powerful sampling-based path planning method, particularly suited for multi-query scenarios where multiple paths between different start and goal pairs are required. PRM operates in two distinct phases:

1. Construction Phase:

    - Sampling: Randomly sample points (nodes) in the free space.
    - Connection: Connect each sampled point to its nearest neighbors, forming a roadmap or graph.
2. Query Phase:

    - Path Finding: Use graph search algorithms (e.g., A*) on the constructed roadmap to find a path from the start to the goal.
## Advantages of PRM
Multi-query Efficiency: Once the roadmap is built, multiple path queries can be efficiently answered.
Scalability: Handles high-dimensional spaces effectively.
Flexibility: Can adapt to various environments and obstacle configurations.
## Algorithm Details
- Construction Phase
    1. Sampling:

        - Generate a set of random, collision-free points in the configuration space.
        - Include the start and goal points in the set of samples.
    2. Connection:

        - For each sampled point, identify its k nearest neighbors.
        - Attempt to connect the point to each neighbor via a straight-line path.
        - Perform collision checking for each potential connection.
        - If the path is collision-free, add an edge between the points in the roadmap.
- Query Phase
    1. Path Finding:
        - Given a start and goal point, locate their nearest neighbors in the roadmap.
        - Add the start and goal to the roadmap if they are not already present.
        - Use a graph search algorithm (e.g., A*) to find the shortest path from start to goal within the roadmap.

We'll implement the PRM algorithm with Pygame for dynamic visualization. The implementation includes sampling, connecting nodes, collision checking, and path finding using A*.

In [17]:
WIDTH, HEIGHT = 800, 800
FPS = 60
OBSTACLE_COUNT = 15
OBSTACLE_RADIUS = 30
ROBOT_RADIUS = 10
SAMPLE_COUNT = 100  
K_NEIGHBORS = 5     
GOAL_THRESHOLD = 20  # Distance threshold to consider the goal reached

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREY = (200, 200, 200)
BLUE = (0, 0, 255)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
ORANGE = (255, 165, 0)
PURPLE = (160, 32, 240)
YELLOW = (255, 255, 0)

In [18]:
def distance(p1, p2):
    return math.hypot(p2[0]-p1[0], p2[1]-p1[1])

def line_intersects_obstacles(p1, p2, obstacles):
    line = LineString([p1, p2])
    for obs in obstacles:
        circle = Point(obs).buffer(OBSTACLE_RADIUS)
        if line.intersects(circle):
            return True
    return False

def get_random_point():
    return (random.randint(50, WIDTH-50), random.randint(50, HEIGHT-50))


In [19]:
def draw_obstacles(obstacles):
    for obs in obstacles:
        pygame.draw.circle(screen, BLACK, obs, OBSTACLE_RADIUS)

def draw_samples(samples):
    for sample in samples:
        pygame.draw.circle(screen, GREY, (int(sample[0]), int(sample[1])), 3)

def draw_connections(connections):
    for conn in connections:
        pygame.draw.line(screen, GREY, conn[0], conn[1], 1)

def draw_path(path):
    if len(path) > 1:
        pygame.draw.lines(screen, ORANGE, False, path, 4)

def draw_robot(position, color=BLUE):
    pygame.draw.circle(screen, color, (int(position[0]), int(position[1])), ROBOT_RADIUS)

def draw_start_goal(start, goal):
    draw_robot(start, GREEN)
    draw_robot(goal, RED)

In [20]:
def get_k_nearest(sample, samples, k):
    distances = []
    for other in samples:
        if other == sample:
            continue
        dist = distance(sample, other)
        distances.append((dist, other))
    distances.sort()
    return [point for (_, point) in distances[:k]]

def reconstruct_path(came_from, start, goal):
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from.get(current)
        if current is None:
            return []  # No path found
    path.append(start)
    path.reverse()
    return path

In [21]:
def dijkstra(graph, start, goal):
    queue = []
    heapq.heappush(queue, (0, start))
    came_from = {}
    cost_so_far = {start: 0}

    while queue:
        current_cost, current = heapq.heappop(queue)

        if current == goal:
            return reconstruct_path(came_from, start, goal)

        for neighbor in graph[current]:
            new_cost = cost_so_far[current] + distance(current, neighbor)
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost
                heapq.heappush(queue, (priority, neighbor))
                came_from[neighbor] = current

    return []  # No path found

In [10]:
class PRMSimulation:
    def __init__(self):
        self.obstacles = self.generate_obstacles()
        self.start = self.get_user_point("Select Start Position")
        self.goal = self.get_user_point("Select Goal Position")
        self.samples = []
        self.graph = {}
        self.connections = []
        self.path = []
        self.construct_prm()

    def generate_obstacles(self):
        obstacles = []
        for _ in range(OBSTACLE_COUNT):
            while True:
                point = get_random_point()
                # Ensure obstacles do not overlap
                overlap = False
                for obs in obstacles:
                    if distance(point, obs) < 2 * OBSTACLE_RADIUS:
                        overlap = True
                        break
                if not overlap:
                    obstacles.append(point)
                    break
        return obstacles

    def get_user_point(self, message):
        font = pygame.font.SysFont(None, 36)
        selected = False
        point = None
        while not selected:
            screen.fill(WHITE)
            text = font.render(message, True, BLACK)
            screen.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2 - text.get_height()//2 - 50))
            instruction = font.render("Click to set position", True, BLACK)
            screen.blit(instruction, (WIDTH//2 - instruction.get_width()//2, HEIGHT//2 - instruction.get_height()//2))
            draw_obstacles(self.obstacles)
            pygame.display.flip()
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    # Ensure the point is not inside an obstacle
                    collision = False
                    for obs in self.obstacles:
                        if distance(pos, obs) < OBSTACLE_RADIUS + ROBOT_RADIUS:
                            collision = True
                            break
                    if not collision:
                        point = pos
                        selected = True
        return point

    def construct_prm(self):
        self.samples = self.sample_free_space(SAMPLE_COUNT)
        self.build_graph()
        # Connect start and goal to the roadmap
        self.connect_start_goal()
        # Find path using Dijkstra's algorithm
        self.path = dijkstra(self.graph, self.start, self.goal)
        if not self.path:
            print("No path found!")

    def sample_free_space(self, count):
        samples = []
        while len(samples) < count:
            p = get_random_point()
            if not self.in_obstacle(p):
                samples.append(p)
        return samples

    def in_obstacle(self, point):
        for obs in self.obstacles:
            if distance(point, obs) <= OBSTACLE_RADIUS:
                return True
        return False

    def build_graph(self):
        self.graph = {sample: [] for sample in self.samples}
        for i, sample in enumerate(self.samples):
            neighbors = get_k_nearest(sample, self.samples, K_NEIGHBORS)
            for neighbor in neighbors:
                if not line_intersects_obstacles(sample, neighbor, self.obstacles):
                    self.graph[sample].append(neighbor)
                    self.connections.append((sample, neighbor))
            if i % 50 == 0:
                self.draw()
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()

    def connect_start_goal(self):
        # Add start and goal to the graph
        self.graph[self.start] = []
        self.graph[self.goal] = []
        all_samples = self.samples + [self.start, self.goal]
        # Connect start
        start_neighbors = get_k_nearest(self.start, all_samples, K_NEIGHBORS)
        for neighbor in start_neighbors:
            if not self.in_obstacle(neighbor) and not line_intersects_obstacles(self.start, neighbor, self.obstacles):
                self.graph[self.start].append(neighbor)
                self.graph[neighbor].append(self.start)
                self.connections.append((self.start, neighbor))
        # Connect goal
        goal_neighbors = get_k_nearest(self.goal, all_samples, K_NEIGHBORS)
        for neighbor in goal_neighbors:
            if not self.in_obstacle(neighbor) and not line_intersects_obstacles(self.goal, neighbor, self.obstacles):
                self.graph[self.goal].append(neighbor)
                self.graph[neighbor].append(self.goal)
                self.connections.append((self.goal, neighbor))

    def draw(self):
        screen.fill(WHITE)
        draw_obstacles(self.obstacles)
        draw_connections(self.connections)
        draw_samples(self.samples)
        draw_start_goal(self.start, self.goal)
        if self.path:
            draw_path(self.path)
        pygame.display.flip()

    def visualize_path(self):
        if self.path:
            for point in self.path:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                self.draw()
                draw_robot(point, BLUE)
                pygame.display.flip()
                clock.tick(60)  
    def run(self):
        self.draw()
        if self.path:
            self.visualize_path()
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
        else:
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()

In [24]:
pygame.init()

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Probabilistic Roadmap (PRM) Simulation")
clock = pygame.time.Clock()

prm_sim = PRMSimulation()
prm_sim.run()


SystemExit: 

---

## Key Differences Between PRM and RRT

1. **Algorithmic Approach:**
   - **PRM:** Uses a two-phase approach (construction and query) to create a reusable roadmap.
   - **RRT:** Grows a tree incrementally from the start to goal, suitable for single-query scenarios.

2. **Data Structure:**
   - **PRM:** Graph-based, with nodes as sampled configurations and edges representing feasible paths.
   - **RRT:** Tree-based, where each node expands the search space incrementally.

3. **Sampling Strategy:**
   - **PRM:** Employs uniform random sampling across the entire space.
   - **RRT:** Uses biased sampling to grow the tree towards unexplored areas or the goal.

4. **Use Cases and Applications:**
   - **PRM:** Ideal for static environments with multiple path queries.
   - **RRT:** Best for dynamic or real-time scenarios requiring fast single-query solutions.

5. **Performance and Scalability:**
   - **PRM:** Scales well in high-dimensional spaces and supports multiple queries efficiently.
   - **RRT:** Faster in finding single paths but may require extensions for optimal solutions.

6. **Path Optimality:**
   - **PRM:** Provides feasible paths; optimality can be improved.
   - **RRT:** Finds feasible paths; RRT* offers improved path optimality.


## Comparison Table

| **Feature**            | **Probabilistic Roadmaps (PRMs)**                      | **Rapidly-exploring Random Trees (RRTs)**               |
|------------------------|--------------------------------------------------------|---------------------------------------------------------|
| **Type**               | Multi-query                                            | Single-query                                            |
| **Data Structure**     | Graph (Roadmap)                                        | Tree                                                    |
| **Sampling Strategy**  | Uniform/random sampling across the space               | Random sampling with possible goal biasing              |
| **Phases**             | Construction and Query phases                          | Incremental tree growth                                 |
| **Use Cases**          | Static environments, multiple path queries             | Dynamic environments, real-time path planning           |
| **Scalability**        | Scales well with high-dimensional spaces               | Efficient exploration but may require modifications for high dimensions |
| **Path Optimality**    | Feasible but not necessarily optimal; can be improved   | Initial paths feasible; optimal variants available      |
| **Reusability**        | High, since the roadmap can serve multiple queries      | Low, primarily designed for single queries              |
| **Performance**        | Higher initial computation, efficient for multiple queries | Faster for single queries, adaptable for real-time       |


---

## When to Use Each Algorithm
### RRT:
- Ideal for scenarios requiring quick, real-time path planning in high-dimensional spaces (e.g., robotic arms).
- Suitable when a single path needs to be found efficiently.
### Spanning Trees (DFS):

- Useful for exhaustive exploration of reachable areas in grid-based environments.
- Suitable for educational purposes to understand graph traversal techniques.
### PRM:

- Best for environments where multiple path queries are required.
- Suitable for complex, cluttered environments where roadmap connectivity is crucial.

---